In [ ]:
"""
*** rerun baseline:

***** COMPLETED:

steps:
  0. extract audiodb.zip files to sinle mp3 files on drive

  1. import csv
  2. get results
      for each csv entry:
        save result of speech-to-latex-logic
  3. give score to all

speech-to-latex-logic:
  audio -> text:  whisper audiodb to get text
  text  -> latex: chatgpt llm of text to get latex

***** POSSIBLE FUTURE TODO:

*** fine-tune llm for maybe better(?) results:
  0. find how to fine-tune model

  1. finetune chatgpt on prompt and answer (take 30% text, 70% train)
  2. run same speech-to-latex-logic with fine-tuned model
  3. give score of all

*** fine-tune whisper with way more data for maybe better(?) results

"""
""

''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
from pathlib import Path

project_folder = Path('/content/drive/My Drive/cs229-audio-project/audiodb_new_simple_eqs/')
audio_path = project_folder / 'audios/'

if not os.path.exists(audio_path):
    zip_file_path = project_folder / 'audios.zip'

    extract_to_path = project_folder

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
else:
    print(f"audio db already extracted to {project_folder}")

audio db already extracted to /content/drive/My Drive/cs229-audio-project/audiodb_new_simple_eqs


In [ ]:
import pandas as pd

data_input = pd.read_csv(project_folder / 'natural_language_augmented.csv', delimiter=';')
display(data_input)

,id,Latex,English 1,English 2,English 3
0,1,\text{{Rank}}(A),The rank of a matrix A.,Rank of the given matrix A.,Matrix A's rank.
1,2,\text{{Nullity}}(A),The nullity of a given matrix A.,Nullity of the matrix A.,Matrix A's nullity.
2,3,p({variable}) = {number} + {number}{variable} ...,P of variable equals number plus number times ...,The probability of a variable is equal to a nu...,"P of variable is equal to a number, plus anoth..."
3,4,\vec{{Force}} = {mass}\vec{{acceleration}},Force vector equals mass times acceleration ve...,The vector of force is equal to the mass times...,"Force, represented as a vector, equals mass mu..."
4,5,E = {mass}{speed\_of\_light}^2,E equals mass times the speed of light squared.,Energy is equal to mass multiplied by the squa...,"E is mass times the speed of light, all squared."
...,...,...,...,...,...
817,818,"[106,800,213] \times [450,206,869]","The cross product of the vectors 106, 800, 213...","The vector 106, 800, 213 crossed with the vect...",The cross product of the vectors with componen...
818,819,"[890,879,829] \times [3381,338,180]","The cross product of the vectors 890, 879, 829...","The vector 890, 879, 829 crossed with the vect...",The cross product of the vectors with componen...
819,820,"[336,739,485] \times [160,546,86]","The cross product of the vectors 336, 739, 485...","The vector 336, 739, 485 crossed with the vect...",The cross product of the vectors with componen...
820,821,"[52,837,748] \times [845,748,168]","The cross product of the vectors 52, 837, 27 a...","The vector 52, 837, 27 crossed with the vector...",The cross product of the vectors with componen...


In [ ]:
data_triplets_df_list = []

for english_version in ("1", "2", "3"):
    triplets = data_input[[f"English {english_version}", "id", "Latex"]].copy()
    triplets["Audio Path"] = triplets.apply(lambda row: audio_path / f"v{english_version}_f{row['id']}.mp3", axis=1)
    triplets.rename(columns={'id': 'Formula ID', f"English {english_version}":"Spoken Text"}, inplace=True)
    triplets["Spoken Text Version"] = english_version

    data_triplets_df_list.append(triplets)
    # print("triplets:")
    # display(triplets)

data_triplets = pd.concat(data_triplets_df_list, ignore_index=True)
display(data_triplets.columns)

Index(['Spoken Text', 'Formula ID', 'Latex', 'Audio Path',
       'Spoken Text Version'],
      dtype='object')

In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
assert torch.cuda.is_available()

In [ ]:
!pip install swifter

In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
import whisper
speech_to_text_model = whisper.load_model("small")
import swifter

In [ ]:
def speech_to_text(audio_file_path):
    model = speech_to_text_model
    result = model.transcribe(str(audio_file_path))
    return result["text"].strip()

In [ ]:
# for each entry in data do: audio -> speech

i = 0
total = len(data_triplets)
print("total:", total)
def wrapper_stt(path):
    # print("looking at", path)
    global i
    i += 1
    if i % 10 == 0:
        print(f"{i}/{total}")
    return speech_to_text(path)

data_triplets['Transcribed Text'] = data_triplets['Audio Path'].apply(wrapper_stt)
print("Done")

total: 2466
10/2466
20/2466
30/2466
40/2466
50/2466
60/2466
70/2466
80/2466
90/2466
100/2466
110/2466
120/2466
130/2466
140/2466
150/2466
160/2466
170/2466
180/2466
190/2466
200/2466
210/2466
220/2466
230/2466
240/2466
250/2466
260/2466
270/2466
280/2466
290/2466
300/2466
310/2466
320/2466
330/2466
340/2466
350/2466
360/2466
370/2466
380/2466
390/2466
400/2466
410/2466
420/2466
430/2466
440/2466
450/2466
460/2466
470/2466
480/2466
490/2466
500/2466
510/2466
520/2466
530/2466
540/2466
550/2466
560/2466
570/2466
580/2466
590/2466
600/2466
610/2466
620/2466
630/2466
640/2466
650/2466
660/2466
670/2466
680/2466
690/2466
700/2466
710/2466
720/2466
730/2466
740/2466
750/2466
760/2466
770/2466
780/2466
790/2466
800/2466
810/2466
820/2466
830/2466
840/2466
850/2466
860/2466
870/2466
880/2466
890/2466
900/2466
910/2466
920/2466
930/2466
940/2466
950/2466
960/2466
970/2466
980/2466
990/2466
1000/2466
1010/2466
1020/2466
1030/2466
1040/2466
1050/2466
1060/2466
1070/2466
1080/2466
1090/2466
1100/2

In [ ]:
from datetime import datetime

def backup_df(df, prefix, folder_path):
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    file_path = folder_path / f"{prefix}_{current_time}.csv"
    df.to_csv(file_path, index=False)

    print(f"DataFrame saved to: {file_path}")

In [ ]:
backup_df(data_triplets, "data_triplets_after_transcribe", project_folder)

DataFrame saved to: /content/drive/My Drive/cs229-audio-project/audiodb_new_simple_eqs/data_triplets_after_transcribe_2024-03-15_14-04-07.csv


In [ ]:
!pip install openai

In [ ]:
# for each entry in data do: transcribed text -> latex

import openai



def text_to_latex_via_llm(english_of_equation):
    # using chatgpt because local llm took *way* too long (15+ min for one)

    preprompt = "take this transcription and write the equation in latex." + \
    "Don't allow any english and use only latex."

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
            {"role": "system", "content" : preprompt},
            {"role": "user", "content" : english_of_equation}
        ]
    )

    latex_from_llm = completion.choices[0].message.content
    return latex_from_llm.strip()

i = 0
total = len(data_triplets)
print("total:", total)
def wrapper_ttl(english):
    # print("looking at", english)
    global i
    i += 1
    if i % 10 == 0:
        print(f"{i}/{total}")
    return text_to_latex_via_llm(english)


total: 2466


In [ ]:
data_triplets['LLM Predicted Latex'] = data_triplets['Transcribed Text'].apply(wrapper_ttl)
print("Done")
backup_df(data_triplets, "data_triplets_after_llm_latex", project_folder)

700/2466
710/2466
720/2466
730/2466
740/2466
750/2466
760/2466
770/2466
780/2466
790/2466
800/2466
810/2466
820/2466
830/2466
840/2466
850/2466
860/2466
870/2466
880/2466
890/2466
900/2466
910/2466
920/2466
930/2466
940/2466
950/2466
960/2466
970/2466
980/2466
990/2466
1000/2466
1010/2466
1020/2466
1030/2466
1040/2466
1050/2466
1060/2466
1070/2466
1080/2466
1090/2466
1100/2466
1110/2466
1120/2466
1130/2466
1140/2466
1150/2466
1160/2466
1170/2466
1180/2466
1190/2466
1200/2466
1210/2466
1220/2466
1230/2466
1240/2466
1250/2466
1260/2466
1270/2466
1280/2466
1290/2466
1300/2466
1310/2466
1320/2466
1330/2466
1340/2466
1350/2466
1360/2466
1370/2466
1380/2466
1390/2466
1400/2466
1410/2466
1420/2466
1430/2466
1440/2466
1450/2466
1460/2466
1470/2466
1480/2466
1490/2466
1500/2466
1510/2466
1520/2466
1530/2466
1540/2466
1550/2466
1560/2466
1570/2466
1580/2466
1590/2466
1600/2466
1610/2466
1620/2466
1630/2466
1640/2466
1650/2466
1660/2466
1670/2466
1680/2466
1690/2466
1700/2466
1710/2466
1720/2466


In [ ]:
!pip install latex2sympy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.7.2-py3-none-any.whl size=140931 sha256=e07c3c2d2df925efd5d620fc45dd2acb0f6baf4ee100f3c2478aa47e3f0cf713
  Stored in directory: /root/.cache/pip/wheels/79/20/ec/30bf7dabc29319ccc0d0c96f910a640513a3c81faa960fed43
Successfully built antlr4-python3-runtime


In [ ]:
from sympy import simplify
from latex2sympy2 import latex2sympy


In [ ]:
# evaluate all of the data!
running_scoring_errors = 0
def score_semantic_similarity(latex1, latex2):
    try:
        # Convert LaTeX to SymPy expressions
        sympy_expr1 = latex2sympy(latex1)
        sympy_expr2 = latex2sympy(latex2)

        # Simplify expressions to their canonical forms
        simplified_expr1 = simplify(sympy_expr1)
        simplified_expr2 = simplify(sympy_expr2)
        # print(f"expr1: {simplified_expr1}, expr2: {simplified_expr2}")

        # Check if the simplified expressions are equivalent
        if simplified_expr1.equals(simplified_expr2):
            return 1  # Expressions are equivalent
        else:
            return 0  # Expressions are not equivalent
    except Exception as e:
        global running_scoring_errors
        running_scoring_errors += 1
        # print(f"Error comparing {latex1} and {latex1}. Error: {e}")
        return None

In [ ]:
i = 0
total = len(data_triplets)
print("total:", total)
def score_wrapper(row):
    global i
    i += 1
    if i % 10 == 0:
        print(f"{i}/{total}")
    return score_semantic_similarity(row["LLM Predicted Latex"], row["Latex"])

total: 2466


In [ ]:
i = 0
running_scoring_errors = 0
data_triplets['Similarity Score'] =  data_triplets[["LLM Predicted Latex", "Latex"]].apply(score_wrapper, axis=1)
print("Done")
print("running_scoring_errors:", running_scoring_errors)
backup_df(data_triplets, "data_triplets_after_scoring", project_folder)
display(data_triplets.dropna(inplace=False))


10/2466
20/2466
30/2466
40/2466
50/2466
60/2466
70/2466
80/2466
90/2466
100/2466
110/2466
120/2466
130/2466
140/2466
150/2466
160/2466
170/2466
180/2466
190/2466
200/2466
210/2466
220/2466
230/2466
240/2466
250/2466
260/2466
270/2466
280/2466
290/2466
300/2466
310/2466
320/2466
330/2466
340/2466
350/2466
360/2466
370/2466
380/2466
390/2466
400/2466
410/2466
420/2466
430/2466
440/2466
450/2466
460/2466
470/2466
480/2466
490/2466
500/2466
510/2466
520/2466
530/2466
540/2466
550/2466
560/2466
570/2466
580/2466
590/2466
600/2466
610/2466
620/2466
630/2466
640/2466
650/2466
660/2466
670/2466
680/2466
690/2466
700/2466
710/2466
720/2466
730/2466
740/2466
750/2466
760/2466
770/2466
780/2466
790/2466
800/2466
810/2466
820/2466
830/2466
840/2466
850/2466
860/2466
870/2466
880/2466
890/2466
900/2466
910/2466
920/2466
930/2466
940/2466
950/2466
960/2466
970/2466
980/2466
990/2466
1000/2466
1010/2466
1020/2466
1030/2466
1040/2466
1050/2466
1060/2466
1070/2466
1080/2466
1090/2466
1100/2466
1110/246

,Spoken Text,Formula ID,Latex,Audio Path,Spoken Text Version,Transcribed Text,LLM Predicted Latex,Similarity Score
30,531 plus 905 equals forty-seven.,31,531 + 905 = 488,/content/drive/My Drive/cs229-audio-project/au...,1,531 plus 905 equals 47,$531 + 905 = 47$,1.0
33,527 plus 97 equals forty-seven.,34,5912 + 97 = 932,/content/drive/My Drive/cs229-audio-project/au...,1,527 plus 97 equals 47,527 + 97 = 47,1.0
34,552 plus 616 equals forty-seven.,35,552 + 616 = 833,/content/drive/My Drive/cs229-audio-project/au...,1,552 plus 616 equals 47,$552 + 616 = 47$,1.0
35,564 plus 560 equals forty-seven.,36,564 + 560 = 8,/content/drive/My Drive/cs229-audio-project/au...,1,564 plus 560 equals 47,$564 + 560 = 47$,1.0
38,25 plus 634 equals forty-seven.,39,486 + 634 = 486,/content/drive/My Drive/cs229-audio-project/au...,1,25 plus 634 equals 47,$$25 + 634 = 47$$,1.0
...,...,...,...,...,...,...,...,...
2202,The smallest multiple that 23 and 24 share is 25.,559,"\text{{LCM}}(861, 6959) = 959",/content/drive/My Drive/cs229-audio-project/au...,3,The smallest multiple that 23 and 24 share is 25,"\text{LCM}(23,24) = 25",0.0
2206,The smallest multiple that 23 and 24 share is 25.,563,"\text{{LCM}}(161, 974) = 245",/content/drive/My Drive/cs229-audio-project/au...,3,The smallest multiple that 23 and 24 share is 25,"\text{LCM}(23,24) = 25",0.0
2208,The smallest multiple that 23 and 24 share is 25.,565,"\text{{LCM}}(793, 832) = 216",/content/drive/My Drive/cs229-audio-project/au...,3,The smallest multiple that 23 and 24 share is 25,"\text{LCM}(23,24)=25",0.0
2212,The smallest multiple that 23 and 24 share is 25.,569,"\text{{LCM}}(743, 348) = 671",/content/drive/My Drive/cs229-audio-project/au...,3,The smallest multiple that 23 and 24 share is 25,"\text{LCM}(23,24) = 25",0.0
